### TFIDF

In [5]:
import pke
from icecream import ic
import pandas as pd
import os
from tqdm import tqdm

def tfidf_extract(text):
    extractor = pke.unsupervised.TfIdf()
    df_counts = pke.load_document_frequency_file(input_file='../data/spark_df.tsv.gz')
    extractor.load_document(input=text, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting(df=df_counts)
    keyphrases = extractor.get_n_best()
    returned_keyphrases = []
    for keyphrase in keyphrases:
        keyphrase = keyphrase[0]
        returned_keyphrases.append(keyphrase)
    return ' '.join(returned_keyphrases)

In [3]:
import ujson, os
from tqdm import tqdm
from icecream import ic

raw_json = '../data/raw/spark.json'
with open('../data/raw/test_spark.txt', 'r') as f:
        test_data = f.readlines()        
test_bug_ids = set()
for bug_id in test_data[1].split():
    test_bug_ids.add(bug_id)
ic(len(test_bug_ids))

compute_df = '../data/compute_df/'
if not os.path.exists(compute_df):
    os.makedirs(compute_df)
    
with open(raw_json, 'r') as f:
    data = f.readlines()
for line in tqdm(data):
    bug = ujson.loads(line)
    bug_id = bug['bug_id']
    if bug_id in test_bug_ids:
        continue
    if not os.path.exists(f'{compute_df}/{bug_id}_summary.txt'):
        with open(f'{compute_df}/{bug_id}_summary.txt', 'w') as f:
            f.write(bug['short_desc'])
        with open(f'{compute_df}/{bug_id}_description.txt', 'w') as f:
            f.write(bug['description'])
    
     

ic| len(test_bug_ids): 2841
100%|██████████| 9579/9579 [00:01<00:00, 5599.36it/s]


In [ ]:
!python3 -m spacy download en_core_web_sm

In [4]:
# !pip install git+https://github.com/boudinfl/pke.git

from pke import compute_document_frequency
from string import punctuation

"""Compute Document Frequency (DF) counts from a collection of documents.

N-grams up to 3-grams are extracted and converted to their n-stems forms.
Those containing a token that occurs in a stoplist are filtered out.
Output file is in compressed (gzip) tab-separated-values format (tsv.gz).
"""

# stoplist for filtering n-grams
stoplist=list(punctuation)

# compute df counts and store as n-stem -> weight values
compute_document_frequency(documents=compute_df,
                           output_file='../data/spark_df.tsv.gz',
                           language='en',                # language of files
                           normalization="stemming",    # use porter stemmer
                           stoplist=stoplist)

In [6]:
project = 'spark'
method = 'tfidf'

df = pd.read_csv('../data/raw/test_{}.csv'.format(project))
tdidf_folder = f'../data/keywords/{project}/tfidf-idf/run_1/'
flag_content_df = pd.read_csv(f'../data/ablation/test_{project}_flag_content.csv')

if not os.path.exists(tdidf_folder):
    os.makedirs(tdidf_folder.format(project))

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    bug_id = row['bug_id']
    if flag_content_df[flag_content_df['bug_id'] == bug_id]['run_flag'].values[0] == 0:
        continue
    if os.path.exists(os.path.join(tdidf_folder, f'{bug_id}.txt')):
        continue
    short_desc = row['short_desc']
    description = row['description']
    short_desc_keywords = tfidf_extract(short_desc)
    description_keywords = tfidf_extract(description)
    
    with open(tdidf_folder.format(project) + '/{}.txt'.format(bug_id), 'w') as f:
        f.write('Summary: {}'.format(short_desc_keywords))
        f.write('\n')
        f.write('Description: {}'.format(description_keywords))
        f.write('\n')  
    

100%|██████████| 2841/2841 [25:13<00:00,  1.88it/s]
